In [ ]:
# Reduce
RDD.reduce(<func>)
사용자가 지정하는 함수를 받아 여러 개의 값을 하나로 줄여줍니다.

In [1]:
from operator import add

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("restaurant-review-average") \
    .master("local[*]") \
    .getOrCreate()
sc = spark.sparkContext

In [5]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [6]:
#RDD에 있는 숫자의 합계를 계산:
sample_rdd.reduce(add)

15

In [7]:
sample_rdd = sc.parallelize([1, 2, 3, 4])
sample_rdd.reduce(lambda x, y : (x * 2)+ y)

18

In [8]:
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

18

In [ ]:
# 파티션을 3개로 지정

In [9]:
sc.parallelize([1, 2, 3, 4], 3).reduce(lambda x, y : (x * 2) + y)

18

In [10]:
sc.parallelize([1, 2, 3, 4], 1).reduce(lambda x, y : (x * 2) + y)

26

# Fold
RDD.fold(zeroValue, <func>)
reduce와 비슷하지만, zeroValue에 넣어놓고 싶은 시작값을 지정해서 reduce가 가능
zeroValue는 파티션 마다 계산이 일어날 때 하나씩 더해지는 값
fold()를 사용하여 합계를 구하되, 초기값을 지정

In [11]:
rdd = sc.parallelize([2, 3, 4], 4)

#숫자의 곱을 계산해서 합한다
print(rdd.reduce(lambda x, y : (x * y))) # 2 x 3 -> x 4
print(rdd.fold(1, lambda x , y : (x * y))) # 1 x 2 x 3 x 4

24
24


In [13]:
rdd.fold(2, lambda x , y : (x * y)) 
# 초기값 2 지정 :: 2x2 , 2x3, 2x4, "" > 4 , 6, 8, "" 
# 다시 초기값 2 지정 :: 4x2*6, 48x2*8, "" > 768

768

In [12]:
print(rdd.reduce(lambda x, y : x + y)) 
print(rdd.fold(1, lambda x, y : x + y)) 

9
14


In [ ]:
# 대규모 숫자 데이터셋 생성
numbers_rdd = sc.parallelize(range(1, 1000001))

# fold를 사용하여 총합계 계산
total_sum = numbers_rdd.fold(0, lambda x, y: x + y)

print(f"Total sum: {total_sum}")

In [ ]:
# 직원 급여 데이터
salaries_rdd = sc.parallelize([50000, 60000, 55000, 75000, 65000, 80000])

# fold를 사용하여 최대 급여 찾기
max_salary = salaries_rdd.fold(0, lambda x, y: max(x, y))

print(f"Maximum salary: {max_salary}")

In [ ]:
# 부서별 직원 급여 데이터
dept_salaries = [("IT", 50000), ("HR", 45000), ("IT", 60000), 
                 ("Finance", 55000), ("HR", 50000), ("IT", 65000)]
dept_rdd = sc.parallelize(dept_salaries)

# foldByKey를 사용하여 부서별 최대 급여 찾기
max_salary_by_dept = dept_rdd.foldByKey(0, lambda x, y: max(x, y))

print("Maximum salary by department:")
for dept, max_salary in max_salary_by_dept.collect():
    print(f"{dept}: {max_salary}")

# GroupBy
RDD.groupBy(<func>)
그룹핑 함수를 받아 reduction
groupBy() 메서드는 주어진 함수를 기준으로 RDD의 요소들을 그룹화합니다.

In [14]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x : x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [15]:
[(x, sorted(y)) for (x, y) in result]

[(0, [2, 8]), (1, [1, 1, 3, 5])]

In [16]:
#판매 데이터 분석 (카테고리별 총 매출 계산)
sales_data = [
    ("Electronics", 1000),
    ("Clothing", 500),
    ("Electronics", 1500),
    ("Books", 300),
    ("Clothing", 750),
    ("Books", 200)
]

In [20]:
# RDD 생성
sales_rdd = sc.parallelize(sales_data)

# 카테고리별 총 매출 계산
category_sales = sales_rdd.groupBy(lambda x: x[0]) \
    .mapValues(lambda values: sum(item[1] for item in values)) \
    .collect()
# 결과 출력
for category, total_sales in category_sales:
    print(f"{category}: ${total_sales}")


Clothing: $1250
Electronics: $2500
Books: $500


In [22]:
#로그 데이터 처리 (IP 주소별 접속 횟수 분석)
# SparkContext 생성 가정 (sc)
log_data = [
    ("192.168.1.1", "login"),
    ("192.168.1.2", "purchase"),
    ("192.168.1.1", "view"),
    ("192.168.1.3", "login"),
    ("192.168.1.2", "login"),
    ("192.168.1.1", "purchase")
]

# RDD 생성
log_rdd = sc.parallelize(log_data)


In [50]:
grouped = log_rdd.groupBy(lambda x: x[0])\
.mapValues(lambda records: len(list(records))).collect()

# 결과 출력
for category, logs in grouped:
    print(f"{category}: {logs}")

192.168.1.1: 3
192.168.1.2: 2
192.168.1.3: 1


In [38]:
#학생 성적 데이터 분석 (과목별 평균 성적 계산)
# SparkContext 생성 가정 (sc)
student_grades = [
    ("Math", 85),
    ("English", 90),
    ("Math", 92),
    ("Science", 88),
    ("English", 95),
    ("Science", 82)
]

# RDD 생성
grades_rdd = sc.parallelize(student_grades)


In [42]:
st_ct = grades_rdd.groupByKey() \
                  .mapValues(lambda values: sum(values) / len(values)) \
                  .collect()
# 결과 출력
for category, avg in st_ct:
    print(f"{category}: {avg}")

Science: 85.0
Math: 88.5
English: 92.5


# 5. aggregate 함수 사용 예제

In [52]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6], numSlices=3)
numbers.collect()

[1, 5, 3, 9, 2, 8, 4, 7, 6]

In [ ]:
# glom()은 각 파티션의 내용을 리스트로 감싸서 반환
# 즉, 파티션 간의 이동이 없고, 내부 자료구조만 바꾸는 연산
# glom()은 RDD의 각 파티션을 배열 형태로 변환하는 narrow transformation

In [53]:
numbers.glom().collect()

[[1, 5, 3], [9, 2, 8], [4, 7, 6]]

In [54]:
numbers = sc.parallelize([1, 5, 3, 9, 2, 8, 4, 7, 6])

In [55]:
# 평균 계산하기
sum_count = numbers.aggregate(
    (0, 0),  # 초기값 (합계, 개수)
    lambda acc, value: (acc[0] + value, acc[1] + 1),  # 각 파티션 내 연산
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # 파티션 간 연산
)
sum_count

(45, 9)

In [56]:
average = sum_count[0] / sum_count[1]
print("Average:", average)  # 5.0

Average: 5.0


In [57]:
# agg 연습
data = ["hello", "world", "spark"]
rdd = sc.parallelize(data)

def seq_op(acc, value):
    text, idx = acc
    return (f"{text} {idx}:{value}".strip(), idx + 1)

def comb_op(acc1, acc2):
    # 줄 번호 이어붙이기: 앞쪽 text 유지 + 뒤쪽 text 뒤에 붙이기
    text1, idx1 = acc1
    text2, idx2 = acc2
    # 줄 번호 충돌 피하려면 idx 조정 필요 (복잡하므로 단순히 붙임)
    return (f"{text1} {text2}".strip(), idx1 + idx2)

In [58]:
zero = ("", 1)

result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

1:hello 1:world 2:spark


In [59]:
rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])
result = rdd.aggregate(zero, seq_op, comb_op)[0]
print(result)

1:spark 2:hadoop 1:ai 2:python 3:sql


In [60]:
# 길이가 5 이상인 문자열 개수 세기
rdd = sc.parallelize(["spark", "hadoop", "ai", "python", "sql"])

result = rdd.aggregate(
    0,
    lambda acc, value: acc + (1 if len(value) >= 5 else 0),
    lambda acc1, acc2: acc1 + acc2
)

print("길이 5 이상인 문자열 개수:", result)

길이 5 이상인 문자열 개수: 3


In [ ]:
#spark.stop()